### IMPORT LIBRARIES

In [1]:
import math
import cv2
import numpy as np
from time import time
import mediapipe as mp
import matplotlib.pyplot as plt

### INITIALIZE POSE DETECTION MODEL

In [2]:
#INITIALIZE MEDIAPIPE POSE CLASS
mp_pose = mp.solutions.pose

#SETTING UP POSE FUNCTION
pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence = 0.3, model_complexity=2)

#INITIALIZING MEDIAPIPE DRAWING CLASS, USEFUL FOR ANNOTATION
mp_drawing = mp.solutions.drawing_utils

# POSE DETECTION ON AN IMAGE

In [3]:
'''

#Read an image from the specific path
sample_img =cv2.imread('images/image1.jpg')

#Specify a size of the figure
plt.figure(figsize = [10,10])

#Display the sample image, also convert BGR to RGB for display
plt.title("Sample Image");plt.axis('off');plt.imshow(sample_img[:,:,::-1]);plt.show()


#Perform pose detection after converting the image into RGB format
results = pose.process(cv2.cvtColor(sample_img, cv2.COLOR_BGR2RGB))

#Check if any landmarks are found
if results.pose_landmarks:
    
    #Iterate two times as we only want to display first two landmarks
    for i in range(2):
        
        #Display the found normalized landmarks
        print(f'{mp_pose.PoseLandmark(i).name}:\n{results.pose_landmark[mp_pose.PoseLandmark(i).value]}')



#Create a copy of the sample image to draw lanmarks om
img_copy = sample_img.copy()

#Check if any landmarks are found 
if reults.pose_landmarks:

    #Draw pose landmarks on the sample image
    mp_drawing.draw_landmarks(image=img_copy, landmarks_list=results.pose_landmarks, connections=mp._pose.POSE_CONNECTIONS)
    
    #Specify a size of the figure
    fig=plt.figure(figsize=[10,10])
    
    #Display the output image with the landmarks drawn, also convert BGR to RGB for display
    plt.title("Output");plt.axis('off');plt.imshow(sample_img[:,:,::-1]);plt.show()
    

    

#Plot pose landmarks in 3D
mp_drawing.plot_landmarks(results.pose_world_landmarks, mp_pose.POSE_CONNECTIONS)


#Read another sample image and perform pose detection on it
image = cv2.imread('images/image1.jpg')
detectPose(image, pose, display = True)
'''

'\n\n#Read an image from the specific path\nsample_img =cv2.imread(\'images/image1.jpg\')\n\n#Specify a size of the figure\nplt.figure(figsize = [10,10])\n\n#Display the sample image, also convert BGR to RGB for display\nplt.title("Sample Image");plt.axis(\'off\');plt.imshow(sample_img[:,:,::-1]);plt.show()\n\n\n#Perform pose detection after converting the image into RGB format\nresults = pose.process(cv2.cvtColor(sample_img, cv2.COLOR_BGR2RGB))\n\n#Check if any landmarks are found\nif results.pose_landmarks:\n    \n    #Iterate two times as we only want to display first two landmarks\n    for i in range(2):\n        \n        #Display the found normalized landmarks\n        print(f\'{mp_pose.PoseLandmark(i).name}:\n{results.pose_landmark[mp_pose.PoseLandmark(i).value]}\')\n\n\n\n#Create a copy of the sample image to draw lanmarks om\nimg_copy = sample_img.copy()\n\n#Check if any landmarks are found \nif reults.pose_landmarks:\n\n    #Draw pose landmarks on the sample image\n    mp_dra

# REAL TIME POSE DETECTION MODEL

### CREATE A POSE DETECTION FUNCTION

In [4]:
def detectPose(image, pose, display = True):
    ''' This performs pose detection for image
    image: input image with a person
    pose: the pose setup function req to perform the pose detection
    display: a boolean value that is if set to true the function displays the original input image, the resultant image, and the pose landmarks in 3D plat and returns nothing.

    output_image: The input image with the detected pose landmarks drawn
    landmarks: A list of detected landmarks converted into their original scale.
    '''
    #Create a copy of the input image
    output_image = image.copy()
    
    #Convert the image from BGR into RGB format
    imageRGB=cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    #Perform the Pose Detection
    results=pose.process(imageRGB)
    
    #Retrive the height and width of the input image
    height, width, _=image.shape
    
    #Initilize a list to store the detected landmarks
    landmarks =[]
    
    #Check if any landmarks are detected
    if results.pose_landmarks:

        
        #draw pose landmarks on the output image
        mp_drawing.draw_landmarks(image=output_image, landmark_list=results.pose_landmarks, connections=mp_pose.POSE_CONNECTIONS)
        
        #iterate over the detected landmarks
        for landmark in results.pose_landmarks.landmark:
            
            #Append the landmarks into the list
            landmarks.append((int(landmark.x*width), int(landmark.y*height), (landmark.z*width)))

    #Check if the original input image and the resultant image are specified to be displayed
    if display:    

        #Display the original input image and the resultant image
        plt.figure(figsize=[22,22])
        plt.subplot(121);plt.imshow(image[:,:,::-1]);plt.title("Original Image");plt.axis('off');
        plt.subplot(122);plt.imshow(output_image[:,:,::-1]);plt.title("Original Image");plt.axis('off');
        
        #Also Plot the pose landmarks in 3D
        mp_drawing.plot_landmarks(results.pose_world_landmarks, mp_pose.POSE_CONNECTIONS)

    #Otherwise
    else:

        #Retuen the output image and the found landmarks
        return output_image, landmarks
        
    
    

### POSE DETECTION ON REAL TIME WEBCAM FEED/VIDEO

In [5]:
#Setup Pose function for video
pose_video = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, model_complexity=1)

#Intilize the VideoCapture object to real from the webcame
video = cv2.VideoCapture(0)

#Create named window for resizing purposes
cv2.namedWindow('Pose Detection', cv2.WINDOW_NORMAL)

#Intialize the videocapture object to read from a video stored in the disk
#video = cv2.VideoCapture('media/running.mp4')

#Set video camera size
video.set(3,1280)
video.set(4,960)

#Initialize a variable to store the time of the previous frame
time1=0

#interate until the video is accessed successfully
while video.isOpened():

    #Read a frame
    ok, frame = video.read()

    #check if frame is not read properly
    if not ok:
        
        #break the loop
        break

    #Flip the frame horizontally for natural (selfie -view) visualization
    frame = cv2.flip(frame, 1)
    
    #Get the width and height of the frame`
    frame_height, frame_width, _ = frame.shape
    
    #Resize the frame while keeping the aspect ratio
    frame= cv2.resize(frame, (int(frame_width * (640/ frame_height)), 640))
    
    #Perfirm Pose Landmark detection
    frame, _ = detectPose(frame, pose_video, display=False)

    #Set time for this frame to the current time
    time2 = time()
    
    #Check if the difference between the previous and this frame time > 0 to avoid division by zero
    if (time2 - time1) > 0:

        #calculate the number of frames per second
        frames_per_second = 1.0 / (time2 - time1)
        
        #write the calculated nimber of frames per second on the frame
        cv2.putText(frame, 'FPS: {}'.format(int(frames_per_second)), (10,30),cv2.FONT_HERSHEY_PLAIN, 2, (0,255,0), 3)

    #Update the previos frame time to this frame time
    #As this frame will become previous frame in next interation
    time1=time2
    
    #Display the frame
    cv2.imshow('Pose Detection', frame)

    #Wait until a key is pressed
    #Retreive the ASCII code of the key pressed
    k=cv2.waitKey(1) & 0xff

    #Check if 'Esc' is pressed
    if (k==27):

        #Break the loop
        break

#Release the VideoCapture
video.release()

#Close the windows
cv2.destroyAllWindows()



C:\Users\Systems\AppData\Local\Programs\Python\Python311\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


# POSE CLASSIFICATION

### with angle heuristics

In [6]:
def calculateAngle(landmark1, landmark2, landmark3):
    ''' this function calculates angle between three different landmarks
    landmark1: the first landmark containg the x,y and z coordinates.
    landmark2: the second landmark containg the x,y and z coordinates.
    landmark3: the third landmark containg the x,y and z coordinates.
    angle: the calculated dangle b/w the three landmarks '''
    #get the req landmarks
    x1, y1, _ = landmark1
    x2, y2, _ = landmark2
    x3, y3, _ = landmark3
    #calculate the angle b/w the three points
    angle = math.degrees(math.atan2(y3-y2, x3-x2) - math.atan2(y1-y2,x1-x2))
    #check if the angle is less than zero
    if angle <0:
        #add 360 to the found angle
        angle +=360
    #return the calculated angle
    return angle

## CREATE A FUNCTION TO PERFORM POSE CLASSIFICATION

### following yoga poses : Warrior II Pose, T Pose, Tree Pose

In [7]:
def classifyPose(landmarks, output_image, display=False):
    ''' this function classifies yoga poses depending upon the angles of various body joints
    landmarks: a list of detected landmarks of the person whose pose needs to be classified
    output_image: a image of the person with the detected pose landmark drawn
    display: a boolean value that is if set to true the function displays the resultant image with the pose label written on it and returns nothing
    output_image: the image with detected pose landmarks drawn and pose label written 
    label: the classified pose label of the person in the output_image. '''
    
    #initialize the label of the pose. if it is not known at this stage
    label ='Unkown Pose'
    #specify the color (Red) with which the label will be written on the image
    color=(0,0,255)

    #Calculate the required angles

    #angle b/w left shoulder, elbow and wrist points
    left_elbow_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value],
                                     landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value],
                                     landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value])
    #angle b/w right shoulder, elbow and wrist points
    right_elbow_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value],
                                     landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value],
                                     landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value])
    #angle b/w left elbow, shoulder and hip points
    left_shoulder_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value],
                                     landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value],
                                     landmarks[mp_pose.PoseLandmark.LEFT_HIP.value])
    #angle b/w right hip, shoulder and elbow points
    right_shoulder_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value],
                                     landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value],
                                     landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value])
    #angle b/w left hip, knee and angle points
    left_knee_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.LEFT_HIP.value],
                                     landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value],
                                     landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value])
    #angle b/w left hip, knee and angle points
    right_knee_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value],
                                     landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value],
                                     landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value])
    
    
    #Check if it is the warrior II pose or T pose
    #As for both of them, both arms should be straight and shoulders should be at the specific angle

    #Check if both arms are straight
    if left_elbow_angle > 165 and left_elbow_angle <195 and right_elbow_angle > 165 and right_elbow_angle <195:
        #check if shoulders are at the req angle
        if left_shoulder_angle > 80 and left_shoulder_angle <110 and right_shoulder_angle > 80 and right_shoulder_angle <110:
    #Check if Warrior II
            #check if one leg straight
            if left_knee_angle >105 and left_knee_angle < 195 or right_knee_angle >105 and right_knee_angle < 195:
                #Check if the other leg is bended at the required angle
                if left_knee_angle >90 and left_knee_angle < 120 or right_knee_angle >90 and right_knee_angle < 120:
                    #specify label of the pose as Warrior II
                    label = 'Warrior II Pose'

    #Check if T Pose
            #Check if both legs are straight
            if left_knee_angle >160 and left_knee_angle < 195 and right_knee_angle >160 and right_knee_angle < 195:
                #specify label of the pose as Warrior II
                    label = 'T Pose'

    # Check if Tree Pose
    # Check if one leg is straight
    if left_knee_angle >165 and left_knee_angle < 195 or right_knee_angle >165 and right_knee_angle < 195:
        #Check if the other leg is bended at req angle
        if left_knee_angle >315 and left_knee_angle < 335 or right_knee_angle >25 and right_knee_angle < 45:
            #specify label of the pose as Warrior II
                    label = 'Tree Pose'

    #Check if pose is classified successfully
    if label != 'Unknown Pose':
        #Update the color(to green) with the label will be written on the image
        color = (0,255,0)

    #Write the label on the output image
    cv2.putText(output_image, label, (10,30), cv2.FONT_HERSHEY_PLAIN, 2, color, 2)

    #Check if the resultant image is specifies to be displayed
    if display:
        #Display the resultant image
        plt.figure(figsize=[10,10])
        plt.imshow(output_image[:,:,::-1]);plt.title("Output Image");plt.axis('off');

    else:
        #Return the output image and the classified label
        return output_image, label
        

    



### POSE CLASSIFICATION ON REAL TIME WEBCAM FEED

In [8]:
#Setup Pose function for video
pose_video = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, model_complexity=1)

#Intialize the VideoCapture object to read from the webcam
camera_video = cv2.VideoCapture(0)
camera_video.set(3,1280)
camera_video.set(4,960)

#Initialize a resizable window
cv2.namedWindow('Pose Classification', cv2. WINDOW_NORMAL)

#Iterate until the webcam is accessed successfully.
while camera_video.isOpened():
    #Read a frame
    ok, frame = camera_video.read()
    #check if frame is not read properly
    if not ok:
        #continue to the next iteration to read the next frame and ignore the empty camera frame
        continue
    #Flip the frame horizontally for natural (selfie-view) viisualization
    frame = cv2.flip(frame, 1)
    #Get the width and height of the frame
    frame_height, frame_width, _ = frame.shape
    #Resize the frame while keeping the aspect ratio
    frame= cv2.resize(frame, (int(frame_width * (640/ frame_height)), 640))
    #Perform Pose Landmark detection
    frame, landmarks = detectPose(frame, pose_video, display=False)

    #Check if landmarks are detected
    if landmarks:
        #Perform the Pose  Classification
        frame, _ = classifyPose(landmarks, frame, display=False)

    #Display the frame
    cv2.imshow('Pose Classification', frame)


    #Wait until a key is pressed
    #Retreive the ASCII code of the key presses
    k = cv2.waitKey(1) & 0xFF

    #Check if 'ESC' is pressed
    if (k== 27):
        #break the line
        break

#Release the VideoCapture object and close the window
camera_video.release()
cv2.destroyAllWindows()
        